In [77]:
class convertCVtoText:
    @staticmethod
    def startConversion(fileName):
        import PyPDF2
        pdfFileObj = open(fileName,'rb')     #'rb' for read binary mode
        pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
        noOfPages=pdfReader.numPages
        text=""
        for pages in range(0,pdfReader.numPages):
            pageObj = pdfReader.getPage(pages)          #'9' is the page number
            text+=pageObj.extractText()
        return text

In [78]:
class regularExp:
    @staticmethod
    def normalizeText(text):
        norText=""
        import re
        norText+= re.sub(r'[^a-zA-Z ]',r' ',text)
        return norText
    @staticmethod
    def Stemmer(text):
        from nltk.stem import PorterStemmer
        import nltk
        words=nltk.word_tokenize(text)
        stemmer = PorterStemmer()
        singles = [stemmer.stem(word) for word in words]
        #print(singles)
        singles=list(set(singles))
        #print(singles)
        stemmedText=" ".join(singles)
        #print(stemmedText)
        return stemmedText
        
        

In [79]:
class NLTKHelper:
    @staticmethod
    def findDocumentMatrix(totalCVText,minFrequency):
        from sklearn.feature_extraction.text import CountVectorizer
        vectorizer=CountVectorizer(stop_words='english',min_df=minFrequency)
        documentMatrix=vectorizer.fit_transform(totalCVText)
        vocabulary=vectorizer.vocabulary_
        return documentMatrix,vocabulary     

In [80]:
class CV:
     def __init__(self,path):
            self.fileName=path
            self.text=convertCVtoText.startConversion(self.fileName)
            self.stemedText=regularExp.Stemmer(self.text)
            self.cleanText=regularExp.normalizeText(self.text)
            self.cleanTextNew=regularExp.normalizeText(self.stemedText)


In [81]:
# class databaseHandler:
#     def __init__(self,password="ST.xavier51",databaseName="cvRanking"):
#         import pymysql
#         try:
#             self.database=pymysql.connect("localhost","root","ST.xavier51","cvRanking")
#             self.cursor = self.database.cursor()
#         except Exception as e:
#             print("error info %s"%e)
#     def entryTheCV(self,data_CV):
#         try:
# #             dataReturned=self.checkDuplication(data_CV[1])
# #             if(len(dataReturned)==0):
#                 add_CV="""insert into recordentry
#                     (checkRecord,recordname)
#                     values(%s,%s)"""
#     #             add_CV="""insert into recordentry
#     #                 (idrecordEntry,checkRecord,recordname)
#     #                 values(%s,%s,%s)"""
#                 #data_CV=("yes","ram")
#                 print(data_CV)
#                 self.cursor.execute(add_CV,data_CV)
#                 self.database.commit()
#                 print("CV name is entered in database")
#         except Exception as e:
#             print("in entryTheCV\t"+str(e))
#     def checkDuplication(self,name_CV):
#         try:
#             check_CV="""select* from recordentry as re where re.recordname=%s"""
#             self.cursor.execute(check_CV,name_CV)
#             result = self.cursor.fetchall()
#         except Exception as e:
#             print("in process checkDuplication\t"+str(e))
#         if(len(result)==0):  
#             return False
#         else:
#             return True  
#     def closeConnection(self):
#         self.database.close()


In [82]:
class CVManager:
    def __init__(self):
        self.CVAdded=None
        #self.database=databaseHandler()
        self.CVList=[]
        self.CVPervoiusFileName=[]
        self.CVFileName=[]
        self.fileNamesWithPath=[]
        self.CVTextColl=[]
        self.CVTextCollTest=[]
        self.documentMatrix=[]
        self.vocabulary={}
        self.documentMatrixTest=[]
        self.vocabularyTest={}
    def list_CVs(self,rootPath):
            import os,json
            try:
                with open("CV coll\\CVNameList.json","r") as file:
                    self.CVPervoiusFileName=json.load(file)
                file.close()
            except (FileNotFoundError, IOError):
                print("CV has not been added yet.\n Adding the CV")
            
            for root, dirs, files in os.walk(rootPath):
                for name in files:
                    self.CVFileName.append(name)
                    with open("CV coll\\CVNameList.json","w") as file:
                        json.dump(self.CVFileName,file)
                    file.close()
                    self.fileNamesWithPath.append(os.path.join(root, name))
    def collectCV(self):
        self.checkCVList()
        if(self.CVAdded):
            for cvPath in self.fileNamesWithPath:
                try:
                    newCV=CV(cvPath)
                    self.CVList.append(newCV)
                    #print("%s named CV is already in database"%cvName)
                except Exception as e:
                    print("in collection of CV \t"+str(e))
    def checkCVList(self):
        if(set(self.CVFileName)& set(self.CVPervoiusFileName)):
            #if common
            print("all cv are added")
            self.CVAdded=False
        else:
            print("adding CV")
            self.CVAdded=True
            
    def collectCVText(self):
        if(self.CVAdded):
            for cv in self.CVList:
                self.CVTextColl.append(cv.cleanText) 
                self.CVTextCollTest.append(cv.cleanTextNew) 
    def findDocumentMatrix(self,minFrequency):
        import json
        try:
            if(self.CVAdded):
                self.collectCVText()
                print("here")
                self.documentMatrix,self.vocabulary=NLTKHelper.findDocumentMatrix(self.CVTextColl,minFrequency)
                self.documentMatrixTest,self.vocabularyTest=NLTKHelper.findDocumentMatrix(self.CVTextCollTest,minFrequency)
                with open("CV coll\\wholeCVText.json","w") as file:
                    json.dump(self.CVTextColl,file)
                file.close()
                with open("CV coll\\wholeCVTextTest.json","w") as file:
                    json.dump(self.CVTextCollTest,file)
                file.close()
            else:
                with open("CV coll\\wholeCVText.json","r") as file:
                    self.CVTextColl=json.load(file)
                file.close()
                with open("CV coll\\wholeCVTextTest.json","r") as file:
                    self.CVTextCollTest=json.load(file)
                file.close()
                self.documentMatrix,self.vocabulary=NLTKHelper.findDocumentMatrix(self.CVTextColl,minFrequency)
                self.documentMatrixTest,self.vocabularyTest=NLTKHelper.findDocumentMatrix(self.CVTextCollTest,minFrequency)
        except Exception as e:
                print("in reading/writing of saved files \t"+str(e))
                
                
            
        
        

In [83]:
manager=CVManager()

In [84]:
manager.list_CVs("CV coll")

In [85]:
manager.collectCV()

all cv are added


In [86]:
manager.findDocumentMatrix(60)


In [87]:
#manager.CVTextColl

In [88]:
manager.vocabulary

{'ability': 0,
 'access': 1,
 'account': 2,
 'accounts': 3,
 'activities': 4,
 'additional': 5,
 'administration': 6,
 'adobe': 7,
 'advanced': 8,
 'agile': 9,
 'ajax': 10,
 'analysis': 11,
 'analyst': 12,
 'android': 13,
 'api': 14,
 'app': 15,
 'application': 16,
 'applications': 17,
 'april': 18,
 'architecture': 19,
 'areas': 20,
 'art': 21,
 'arts': 22,
 'asp': 23,
 'assistant': 24,
 'assisted': 25,
 'associate': 26,
 'august': 27,
 'authorized': 28,
 'bachelor': 29,
 'backend': 30,
 'based': 31,
 'basic': 32,
 'best': 33,
 'budget': 34,
 'build': 35,
 'building': 36,
 'built': 37,
 'business': 38,
 'ca': 39,
 'cases': 40,
 'center': 41,
 'certifications': 42,
 'change': 43,
 'changes': 44,
 'client': 45,
 'clients': 46,
 'code': 47,
 'college': 48,
 'com': 49,
 'communication': 50,
 'communications': 51,
 'community': 52,
 'companies': 53,
 'company': 54,
 'complete': 55,
 'complex': 56,
 'compliance': 57,
 'components': 58,
 'computer': 59,
 'concepts': 60,
 'conducted': 61,
 'c

In [89]:
manager.vocabularyTest

{'abil': 0,
 'access': 1,
 'account': 2,
 'achiev': 3,
 'action': 4,
 'activ': 5,
 'ad': 6,
 'addit': 7,
 'administr': 8,
 'adob': 9,
 'advanc': 10,
 'advertis': 11,
 'agil': 12,
 'ajax': 13,
 'allow': 14,
 'analysi': 15,
 'analyst': 16,
 'analyt': 17,
 'analyz': 18,
 'android': 19,
 'ani': 20,
 'anywher': 21,
 'api': 22,
 'app': 23,
 'appli': 24,
 'applic': 25,
 'approv': 26,
 'april': 27,
 'architectur': 28,
 'area': 29,
 'art': 30,
 'asp': 31,
 'assess': 32,
 'assign': 33,
 'assist': 34,
 'associ': 35,
 'assur': 36,
 'audit': 37,
 'august': 38,
 'author': 39,
 'autom': 40,
 'award': 41,
 'bachelor': 42,
 'backend': 43,
 'bas': 44,
 'base': 45,
 'basic': 46,
 'best': 47,
 'board': 48,
 'brand': 49,
 'budget': 50,
 'build': 51,
 'built': 52,
 'busi': 53,
 'ca': 54,
 'case': 55,
 'center': 56,
 'certif': 57,
 'certifi': 58,
 'chang': 59,
 'check': 60,
 'class': 61,
 'client': 62,
 'close': 63,
 'code': 64,
 'collabor': 65,
 'collect': 66,
 'colleg': 67,
 'com': 68,
 'commun': 69,
 'com

In [90]:
# for(k,v), (k2,v2) in zip(manager.vocabulary.items(), manager.vocabularyTest.items()):
#     print(k,v)